In [1]:
!pip install tensorflow pandas scikit-learn



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
#from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import LSTM, Dropout, Dense, Conv1D, MaxPooling1D
from keras.optimizers import Adam
from keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import EarlyStopping

# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from keras.optimizers import Adam
# from keras.losses import Huber
# from tensorflow.keras.callbacks import EarlyStopping

pd.set_option('display.max_columns',None)

In [50]:
def call_rmsle(y_true, y_pred):
    # Ensure predictions and true values are positive by adding 1
    y_true = tf.math.maximum(y_true, 0)
    y_pred = tf.math.maximum(y_pred, 0)

    # Compute the logarithm of the true and predicted values
    log_true = tf.math.log1p(y_true)
    log_pred = tf.math.log1p(y_pred)

    # Compute the squared difference
    squared_diff = tf.math.square(log_true - log_pred)

    # Compute the mean of the squared differences
    mean_squared_diff = tf.math.reduce_mean(squared_diff)

    # Return the square root of the mean squared differences
    return tf.math.sqrt(mean_squared_diff)

def remove_leading_zeros(df):
    # Find the index of the first non-zero sales value
    first_non_zero_index = df['sales'].ne(0).idxmax()

    # Filter the DataFrame to remove leading zeros
    return df.loc[first_non_zero_index:]

def average_absolute_percentage_difference(y_test_actual, lstm_predictions_actual):
    """
    Calculate the average absolute percentage difference (APD) between actual and predicted values,
    ignoring cases where actual values are zero to avoid division by zero.

    Parameters:
    y_test_actual (array-like): Array of actual values.
    lstm_predictions_actual (array-like): Array of predicted values.

    Returns:
    float: The average absolute percentage difference.
    """
    # Convert inputs to numpy arrays if they are not already
    y_test_actual = np.array(y_test_actual)
    lstm_predictions_actual = np.array(lstm_predictions_actual)
    
    # Create a boolean mask where y_test_actual is not zero
    non_zero_mask = y_test_actual != 0
    
    # Apply the mask to filter out zero values
    filtered_y_test_actual = y_test_actual[non_zero_mask]
    filtered_predictions = lstm_predictions_actual[non_zero_mask]
    
    # Check if there are any non-zero values left
    if len(filtered_y_test_actual) == 0:
        raise ValueError("All values in y_test_actual are zero; cannot compute APD.")
    
    # Calculate the absolute percentage difference
    abs_pct_dff = (np.abs(filtered_y_test_actual - filtered_predictions) / filtered_y_test_actual) * 100
    
    # Calculate and return the mean of the absolute percentage differences
    return abs_pct_dff.mean()

In [4]:
df_train_all    = pd.read_csv('data_dir/wip_data/preprocessed_train_data_v1.csv',index_col=0)


In [5]:
df_test_all    = pd.read_csv('data_dir/wip_data/preprocessed_test_data_v1.csv',index_col=0)

In [47]:
df_lstm_results  = pd.DataFrame()
val_results = pd.DataFrame(columns=['Store_Number', 'Family', 'RMSLE','abs_pct_diff'])

In [7]:
all_store_nr_all   = df_train_all['store_nbr'].unique()
all_store_nr_all   = np.sort(all_store_nr_all)

In [41]:
all_store_nr       = all_store_nr_all[47:54]


In [51]:
for store_nr in (all_store_nr):
    all_family =   df_train_all[df_train_all['store_nbr']==store_nr]['family'].unique()  
    for fam in all_family:
        if df_train_all[(df_train_all['store_nbr']==store_nr) & (df_train_all['family']==fam)]['sales'].sum() > 0 :
            print("Running for store ", store_nr," and family ",fam)
            
            df_train    = df_train_all[(df_train_all['store_nbr']==store_nr) & (df_train_all['family']==fam)]
            df_test     = df_test_all[(df_test_all['store_nbr']==store_nr) & (df_test_all['family']==fam)]
            ###############################################################################
            ##########       Some Preprocessing is required     ###########################
            ###############################################################################

            df  = df_train.copy()

            #removing starting sales = 0 rows
            df = remove_leading_zeros(df)

            # Convert month and day to cyclical features
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
            df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

            df['wom_sin'] = np.sin(2 * np.pi * df['week_of_month'] / 5)
            df['wom_cos'] = np.cos(2 * np.pi * df['week_of_month'] / 5)

            df['week_sin'] = np.sin(2 * np.pi * df['week_number'] / 52)
            df['week_cos'] = np.cos(2 * np.pi * df['week_number'] / 52)


            df['is_weekend'] = df['day_of_week'].isin([0, 6]).astype('int')
            df['is_midweek'] = df['day_of_week'].isin([3, 4]).astype('int')

            #df = df[df['sales']>0] # as we created a function to remove leading 0
            df = df[df['IS_HOLIDAY']==0]
            df['year'] = df['year']-df['year'].min()

            ###########################################################################################
            ###############     Preparing for LSTM              ######################################
            #########################################################################################

            # Features to be scaled #removed 'holiday_type',
            features_to_scale = [ 'oil_price', 'year']
            # Features to be left unscaled (those ending with 'sin' or 'cos') # removed 'IS_HOLIDAY'
            features_to_leave = ['onpromotion',   'month_sin','month_cos',
                     'day_sin','day_cos',  'wom_sin','wom_cos', 'week_sin','week_cos','is_weekend','is_midweek' ]

            features = features_to_scale + features_to_leave
            target = 'sales'

            # Splitting data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, shuffle=False)
            
            ##########Experiment: Allowing data leakage#######
            X_train = df[features]
            y_train = df[target]
            ###################################################
            
            # Initialize scalers
            feature_scaler = MinMaxScaler(feature_range=(0, 1))
            target_scaler = MinMaxScaler(feature_range=(0, 1))

            # Scale only the selected features
            X_train_scaled_part = feature_scaler.fit_transform(X_train[features_to_scale])
            X_test_scaled_part = feature_scaler.transform(X_test[features_to_scale])

            # Combine scaled features with unscaled features (still as NumPy arrays)
            X_train_scaled = np.hstack((X_train_scaled_part, X_train[features_to_leave].values))
            X_test_scaled = np.hstack((X_test_scaled_part, X_test[features_to_leave].values))

            # Scale target variable
            y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
            y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))

            # Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
            timesteps = 1  # You need to define this based on your data and model requirements
            X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
            X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))
            
            #############################################################################################
            #################                Train the model                         ####################
            #############################################################################################

            # Build your CNN-LSTM model
            model = Sequential()

            # Add a Conv1D layer
            model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(timesteps, X_train_reshaped.shape[2])))
            model.add(MaxPooling1D(pool_size=1))

            # Add LSTM layers (directly after Conv1D without flattening)
            model.add(LSTM(256, return_sequences=True))
            model.add(Dropout(0.2))
            model.add(LSTM(100))
            model.add(Dropout(0.2))

            # Add the output layer
            model.add(Dense(1))

            # Define early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

            # Compile the model with RMSLE as the loss function
            optimizer = Adam(learning_rate=0.001)
            model.compile(optimizer=optimizer, loss=call_rmsle)

            # Train the model
            history = model.fit(X_train_reshaped, y_train_scaled, epochs=100, 
                                validation_data=(X_test_reshaped, y_test_scaled), batch_size=32, callbacks=[early_stopping])


#             # Build your LSTM model
#             model = Sequential()
#             model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
#             model.add(Dropout(0.2))
#             model.add(LSTM(100))
#             model.add(Dropout(0.2))

#             model.add(Dense(1))

#             # Define early stopping
#             early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#             # Compile the model with RMSLE as the loss function
#             optimizer = Adam(learning_rate=0.001)
#             model.compile(optimizer=optimizer, loss=call_rmsle,metrics = ['accuracy'] )

#             # Train the model
#             history = model.fit(X_train_reshaped, y_train_scaled, epochs=30, 
#                                 validation_data=(X_test_reshaped, y_test_scaled), batch_size=32,callbacks=[early_stopping])

            ##############################################################################################
            #########################          Make Predictions              #############################
            ##############################################################################################


            # Predictions
            lstm_predictions = model.predict(X_test_reshaped)

            # Ensure both predictions and actual values are of type float64
            y_test_scaled = y_test_scaled.reshape(-1, 1).astype('float64')
            lstm_predictions_scaled = lstm_predictions.astype('float64')

            # Calculate RMSLE
            lstm_rmsle_val = call_rmsle(y_test_scaled, lstm_predictions_scaled)

            y_test_actual = target_scaler.inverse_transform(y_test_scaled).astype('float64')
            lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')
            lstm_rmsle_val_actual = call_rmsle(y_test_actual, lstm_predictions_actual)
            
         
            abs_pct_dff      = average_absolute_percentage_difference(y_test_actual, lstm_predictions_actual)
            print("RSMLE we got for store number ",store_nr," and family ",fam," is ",lstm_rmsle_val_actual)
            print("Abs pct we got for store number ",store_nr," and family ",fam," is ",abs_pct_dff)
            
            val_results = val_results.append({'Store_Number': store_nr, 'Family': fam, 'RMSLE': lstm_rmsle_val_actual, 'abs_pct_diff':abs_pct_dff}, ignore_index=True)

            ##############################################################################################
            #########################             Applying model on test data       ######################
            ##############################################################################################

            del df
            df   = df_test.copy()

            ###############################################################################
            ##########       test_data_preprocessing            ###########################
            ###############################################################################



            # Convert month and day to cyclical features
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
            df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

            df['wom_sin'] = np.sin(2 * np.pi * df['week_of_month'] / 5)
            df['wom_cos'] = np.cos(2 * np.pi * df['week_of_month'] / 5)

            df['week_sin'] = np.sin(2 * np.pi * df['week_number'] / 52)
            df['week_cos'] = np.cos(2 * np.pi * df['week_number'] / 52)

            df['is_weekend'] = df['day_of_week'].isin([0, 6]).astype('int')
            df['is_midweek'] = df['day_of_week'].isin([3, 4]).astype('int')

            df['year'] = df['year']-df_train['year'].min()

            ###########################################################################################
            ###############     Preparing for LSTM              ######################################
            #########################################################################################

            # Features to be scaled #removed 'holiday_type',
            features_to_scale = [ 'oil_price', 'year']
            # Features to be left unscaled (those ending with 'sin' or 'cos') # removed 'IS_HOLIDAY',
            features_to_leave = ['onpromotion',   'month_sin','month_cos',
                     'day_sin','day_cos',  'wom_sin','wom_cos', 'week_sin','week_cos','is_weekend','is_midweek' ]

            features = features_to_scale + features_to_leave

            # Scale only the selected features
            X_test_scaled_part = feature_scaler.transform(df[features_to_scale])

            # Combine scaled features with unscaled features (still as NumPy arrays)
            X_test_scaled = np.hstack((X_test_scaled_part, df[features_to_leave].values))

            X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

            # Predictions
            lstm_predictions = model.predict(X_test_reshaped)
            lstm_predictions_scaled = lstm_predictions.astype('float64')

            lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')

            # Add predictions to the dataframe
            df['predicted_sales'] = lstm_predictions_actual

            this_df   = df[['id','predicted_sales']]

            df_lstm_results = pd.concat([df_lstm_results,this_df])

            del this_df
            del df

Running for store  3  and family  10


C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.2128 - val_loss: 0.0627
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0592 - val_loss: 0.0534
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0577 - val_loss: 0.0528
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0548 - val_loss: 0.0545
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0557 - val_loss: 0.0562
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0561 - val_loss: 0.0537
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0539 - val_loss: 0.0549
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0544 - val_loss: 0.0555
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
RSMLE we got for store number  3  and family  10  is  tf.Tensor(0.39497016971843524, shape=(), dtype=float64)
Abs pct we got for store number  3  and family  10  is  11.382443877403524
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


C:\Users\Monil\AppData\Local\Temp\ipykernel_98436\4265401761.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_results = val_results.append({'Store_Number': store_nr, 'Family': fam, 'RMSLE': lstm_rmsle_val_actual, 'abs_pct_diff':abs_pct_dff}, ignore_index=True)


In [46]:
df_lstm_results['predicted_sales'] = np.where(df_lstm_results['predicted_sales'] < 0, 0, df_lstm_results['predicted_sales'])
df_lstm_results.tail()

,id,predicted_sales
3021226,3021226,610.533986
3023008,3023008,453.617556
3024790,3024790,449.423477
3026572,3026572,422.325766
3028354,3028354,420.313819


In [10]:
df_lstm_results.to_csv('data_dir/output_data/lstm_prediction_submission_47_54.csv',index=False)